# Stable Diffusion txt2img with prompt_correction

## 機能

* Stable DIffusion の txt2img による画像生成
  * diffusers 依存なし
* [prompt_correction patch](https://gist.github.com/td2sk/33044aabd8cc080137bceb3f87f650ef) 適用済み

## 使い方

1. ランタイムを GPU に設定
1. モデルファイル(sd-v1-4.ckpt)を Google Drive に配置
1. 設定 の項目にあるパスをモデルファイル配置先
1. 1 ~ 4 を実行して環境構築する
1. 5 を実行して画像を生成する

パラメータを変更して画像を再作成する場合は 5 のみ再実行すれば良い

## 更新履歴

* 2022/09/07 公開

In [1]:
#@markdown # 1. 設定

import os

#@markdown Google Drive マウント先
MOUNT_PATH = "/content/googledrive" #@param {type: "string"}

#@markdown モデルファイル 
MODEL_PATH = "/content/googledrive/MyDrive/stable-diffusion/models/sd-v1-4.ckpt" #@param {type: "string"}

# 各種ライブラリの配置パス
SD_PATH = "/content/"

# Stable Diffusion model config path
MODEL_CONFIG_PATH = os.path.join(SD_PATH, "stable-diffusion", "configs", "stable-diffusion", "v1-inference.yaml")

In [ ]:
#@markdown # 2. Google Drive をマウント
from google.colab import drive
drive.mount(MOUNT_PATH)

In [ ]:
#@markdown # 3. ライブラリの取得

# ソースコード取得
%cd $SD_PATH
!git clone https://github.com/td2sk/stable-diffusion
!git clone https://github.com/CompVis/taming-transformers.git
!git clone https://github.com/openai/CLIP.git

import sys
sys.path.append('./stable-diffusion')
sys.path.append('./taming-transformers')
sys.path.append('./CLIP')

# pipライブラリ取得
!pip install albumentations==0.4.3 opencv-python==4.1.2.30 pudb==2019.2 imageio==2.9.0 imageio-ffmpeg==0.4.2 pytorch-lightning==1.4.2 omegaconf==2.1.1 test-tube>=0.7.5 streamlit>=0.73.1 einops==0.3.0 torch-fidelity==0.3.0 transformers==4.19.2 torchmetrics==0.6.0 kornia==0.6 ftfy

# 不要なライブラリを削除
!pip uninstall -y torchtext

In [ ]:
#@markdown # 4. モデルの読み込み
#@markdown 読み込んだモデルを model に代入する
#@markdown * Colab ランタイム起動時に1度実行すればよい
from scripts.txt2img_nb import load_model, main
model = load_model(config=MODEL_CONFIG_PATH, ckpt=MODEL_PATH)

In [ ]:
#@markdown # 5. 実行

#@markdown ## 利用方法
#@markdown 
#@markdown main メソッドに model と prompt を渡して画像を生成する。
#@markdown 
#@markdown main メソッドは Stable Diffusion の txt2img.py のコマンドラインオプションと同名の引数をサポートしている。
#@markdown 
#@markdown ```
#@markdown model                                  : 手順4 (モデルの読み込み) で設定した変数を指定
#@markdown prompt (str)                           : txt2img のプロンプト
#@markdown ddim_steps (int, optional)             : default=50
#@markdown fixed_code (boolean, optional)         : default=False
#@markdown plms (boolean, optional)               : default=False
#@markdown ddim_eta (float, optional)             : default=0.0
#@markdown n_iter (int, optional)                 : default=1
#@markdown H (int, optional)                      : default=512,
#@markdown W (int, optional)                      : default=512
#@markdown C (int, optional)                      : default=4
#@markdown f (int, optional)                      : default=8
#@markdown n_samples (int, optional)              : deafult=1
#@markdown n_rows (int, optional)                 : default=0
#@markdown scale (float, optional)                : default=7.5
#@markdown seed (int, optional)                   : default=42
#@markdown precision (str, optional)              : default="autocast"
#@markdown prompt_correction (List[str], optional): default=[]
#@markdown ```

# パラメーターの設定例
prompt = "pyramid"
prompt_correction =["egypt::-0.20", "japan::0.20"]
seed = 751
scale = 7.5
plms=True

# 重みを変えながら複数画像を生成する場合は 以下のように f-string を使うとよい
# for weight in range(20):
#   prompt_correction = [f"egypt::{-i/100}", f"japan::{i/100}"]
#   (その他コード)

# main メソッドで画像を生成する
images = main(model, prompt, seed=seed, scale=scale, plms=plms, prompt_correction=prompt_correction)
images[0]